# Sheet01
## Loading the data

In [3]:
from sklearn.datasets import load_digits

digits = load_digits()

data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

## Extract necessary data

In [4]:
import numpy as np

X = []
y = []
for i in range(target.shape[0]):
    if target[i]==3:
        X.append(data[i])
        y.append(1)
    elif target[i]==8:
        X.append(data[i])
        y.append(-1)
    else: continue

y = np.array(y)
X = np.array(X)
ones = np.ones(X.shape[0],dtype=float)
ones = ones[:, np.newaxis]
X = np.concatenate((X, ones),axis=1)
print("Shape of X is:", X.shape)
print("Shape of y is:", y.shape)

Shape of X is: (357, 65)
Shape of y is: (357,)


## Logistic Regression and Cross Validation

In [5]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

scores = []
C_list = np.geomspace(1e-4,1e2,100)
fig, ax = plt.subplots()

for lamb_inv in C_list:
    clf1 = LogisticRegression(C = lamb_inv, fit_intercept = False, random_state = 0, solver = "liblinear").fit(X_train,y_train)
    plt.hold = True
    ax.plot(lamb_inv, clf1.score(X_test,y_test),"ro")
    scores.append(clf1.score(X_test,y_test))

scores = np.array(scores)
lamb = 1/C_list[np.argmax(scores)]
reg_param = lamb
plt.show()
print("The maximum score",np.max(scores),"is at lamb =",lamb)

<Figure size 640x480 with 1 Axes>

The maximum score 1.0 is at lamb = 231.0129700083158


## 1.2 Optimization Methods

In [6]:
def sigmoid(z):
    sigma =  1/(1+np.exp(-z))
    return sigma


def gradient(beta, X, y):
    update = np.zeros(beta.shape[0])
    for i in range(X.shape[0]):
        update = update + sigmoid(y[i]*np.dot(X[i,:],beta)) * y[i] * X[i,:]
    grad = (1/reg_param) * beta + (1/y.shape[0]) * update
    return grad

def gradient_achita(beta, X, y):
    N = y.shape[0]
    temp = sigmoid(-y*np.dot(X,beta))*y
    #Here is the correction of the grad
    result = beta - 1.0/N*(np.dot(X.T,temp))
    return result

def predict(beta,X):
    decision = np.dot(X,beta)
    y_computed = np.sign(decision)
    return y_computed

def zero_ones_loss(y_prediction, y_truth):
    zero_ones_vec = 1/2 * (y_prediction - y_truth)
    return np.dot(zero_ones_vec , zero_ones_vec)
    

## Algorithms

In [7]:
def gradient_descent(m,beta,X,y,tau):
    for i in range(m):
        beta = beta - tau * gradient(beta,X,y)
    return beta

def stochastic_gradient(m,beta,X,y,tau0,gamma):
    tau = tau0
    for i in range(m):
        tau = tau/(1+np.power(gamma,i))
        X_random = X[np.random.randint(0,X.shape[0]),:].reshape(1,65)
        beta = beta - tau * gradient(beta, X_random, y)
    return beta

def SG_mini_batch(m,beta,X,y,tau0,gamma,B):
    tau = tau0
    for i in range(m):
        tau = tau/(1+gamma*i)
        rows_random = np.random.choice(np.arange(X.shape[0]),B,replace = True)
        X_random = X[rows_random,:]
        beta = beta - tau * gradient(beta,X_random,y)
    return beta

def SG_momentum(m,beta,X,y,tau0,gamma,mu):
    g = np.zeros(65)
    tau = tau0
    for i in range(m):
        tau = tau/(1+gamma*i)
        X_random = X[np.random.randint(0,X.shape[0]),:].reshape(1,65)
        g = mu*g +(1-mu)*gradient(beta,X_random,y)
        beta = beta - tau*g
    return beta

def ADAM(m,beta,X,y,tau):
    g = 0
    q = 1
    mu1 = 0.9
    mu2 = 0.999
    epsilon = 1e-8
    for i in range(m):
        X_random = X[np.random.randint(0,X.shape[0]),:].reshape(1,65)
        grad = gradient(beta,X_random,y)     
        g = mu1*g + (1-mu1)*grad
        q = mu2*q + (1-mu2)*(grad*grad)
        g_tilt = g/(1-np.power(mu1,i+1))
        q_tilt = q/(1-np.power(mu2,i+1))  
        beta = beta - tau/(np.sqrt(q_tilt)+epsilon) * g_tilt
    return beta

def SAG(m,beta,X,y,tau0,gamma):
    g_stored = []
    tau = tau0
    for i in range(y.shape[0]):
        g_stored.append( y[i]* X[i] * sigmoid(-y[i] * np.dot( X[i] , beta)))
    g_stored = np.array(g_stored)
    g = np.sum(g_stored,axis = 0)/y.shape[0]
    for j in range(m):
        tau = tau/(1+gamma*i)
        i = np.random.randint(0,g_stored.shape[0])
        g_update = y[i] * np.dot( X[i] , sigmoid(-y[i]* np.dot(X[i] , beta)))
        g = g + 1/X.shape[0] *(g_update - g_stored[i])
        g_stored[i] = g_update
        beta = beta * (1- tau/reg_param) - tau*g 
    return beta


def DCA(m,X,y):
    alpha = np.random.rand(y.shape[0])    
    beta = reg_param/y.shape[0] * np.dot( alpha * y, X)
    for j in range(m):
        i = np.random.randint(alpha.shape[0])
        f1 = y[i]*np.dot(X[i],beta) + np.log(alpha[i]/(1-alpha[i]))      
        f2 = reg_param/y.shape[0] * np.dot(X[i],X[i]) + 1/(alpha[i]*(1-alpha[i]))       
        alpha_new = alpha[i] - f1/f2
        beta = beta + (reg_param/y.shape[0]) *y[i]* X[i] * (alpha_new - alpha[i])
    return beta


def newton_raphson(m,beta,X,y):
    for t in range(m):
        D = X.shape[1]
        N = X.shape[0]
        beta = np.zeros(D)
        z = np.dot( X , beta)
        y_tilt = y/sigmoid(np.dot(y,z))
        diag = np.array([reg_param/N*sigmoid(z[i])*sigmoid(-z[i]) for i in range(N)])
        W = np.diagflat(diag)
        beta = np.dot( np.dot( np.linalg.inv( np.identity(D) + np.dot ( np.dot(X.T , W) , X )) , np.dot(X.T,W) ), z + y_tilt)
    return beta

## 1.3 Comparison

In [9]:
N = y_train.shape[0]
D = X.shape[1]
beta0 = np.zeros(D)
g0 = 0
alpha0 = np.random.uniform(size = N)
tau0 = 0.1
mu = 0.1
gamma = 0.01
B = 20
print("GD,tau0", zero_ones_loss(predict( gradient_descent(150, beta0 ,X_train ,y_train ,tau0) ,X_test),y_test))
print("SG" , zero_ones_loss(predict( stochastic_gradient(150, beta0 ,X_train ,y_train ,tau0,gamma) ,X_test),y_test))
print("SG_mini_", zero_ones_loss(predict( SG_mini_batch(150, beta0 ,X_train ,y_train ,tau0,gamma,B) ,X_test),y_test))
print("SG_momentum",zero_ones_loss(predict( SG_momentum(150, beta0 ,X_train ,y_train ,tau0,gamma,mu) ,X_test),y_test))
print("ADAM", zero_ones_loss(predict( ADAM(150, beta0 ,X_train ,y_train ,tau0) ,X_test),y_test))
print("SAG", zero_ones_loss(predict( SAG(150, beta0 ,X_train ,y_train ,tau0,gamma) ,X_test),y_test))

print("DCA", zero_ones_loss(predict( DCA(150,X_train ,y_train) ,X_test),y_test))
print("Newton/Raphson",zero_ones_loss(predict( newton_raphson(150,beta0,X_train ,y_train) ,X_test),y_test))

GD,tau0 107.0
SG 57.0
SG_mini_ 55.0
SG_momentum 57.0
ADAM 57.0
SAG 106.0
DCA 55.0
Newton/Raphson 0.0


## Finding the best hyperparameters

In [17]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 10)

TAU = np.array([1e-3,1e-2,0.1])
MU = np.array([0.1,0.2,0.5])
GAMMA = np.array([1e-4,1e-3,1e-2])

        
GD_HP = np.array([TAU,[0,0,0]])
SGD_HP = np.zeros((3,3))
SGMBatch_HP = np.zeros((3,3))
SGMomentum_HP = np.zeros((3,3,3))
ADAM_HP = np.array([TAU,[0,0,0]])
SAG_HP = np.zeros((3,3))

for tau_index in range(3):
    tau0 = TAU[tau_index]
    sum_GD = 0
    sum_ADAM = 0
    for train_index, test_index in kf.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        sum_GD += zero_ones_loss(predict(gradient_descent(10,beta0,X_train,y_train,tau0),X_test),y_test)
        sum_ADAM += zero_ones_loss(predict(ADAM(10,beta0,X_train,y_train,tau0),X_test),y_test)
    GD_HP[1,tau_index] = sum_GD
    ADAM_HP[1,tau_index] = sum_ADAM
    for gamma_index in range(3):
        gamma = GAMMA[gamma_index]
        sum_SG = 0
        sum_SGMBatch = 0
        sum_SAG = 0
        for train_index, test_index in kf.split(X,y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            sum_SG +=  zero_ones_loss(predict(stochastic_gradient(150,beta0,X_train,y_train,tau0,gamma),X_test),y_test)
            sum_SGMBatch += zero_ones_loss(predict(SG_mini_batch(150,beta0,X_train,y_train,tau0,gamma,20),X_test),y_test)
            sum_SAG += zero_ones_loss(predict(SAG(150,beta0,X_train,y_train,tau0,gamma),X_test),y_test)
        SGD_HP[tau_index,gamma_index] = sum_SG
        SGMBatch_HP[tau_index,gamma_index] = sum_SGMBatch
        SAG_HP[tau_index,gamma_index] = sum_SAG
        for mu_index in range(3):
            mu = MU[mu_index]
            sum_SGMomentum = 0
            for train_index, test_index in kf.split(X,y):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                sum_SGMomentum += zero_ones_loss(predict(SG_momentum(150,beta0,X_train,y_train,tau0,gamma,mu),X_test),y_test)
            SGMomentum_HP[tau_index,gamma_index,mu_index] = sum_SGMomentum


/home/lorenz/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [51]:
print("For gradient descent the best learning rate is", GD_HP[0,np.argmin(GD_HP[1,:])])
print("For ADAM the best learning rate is",ADAM_HP[0,np.argmin(ADAM_HP[1,:])])
print(SGD_HP)
print("For stochastic GD the best learning rate is", TAU[np.argmin(SGD_HP)//3])
print("For stochastic mini batch the best learning rate is", TAU[np.argmin(SGMBatch_HP)//3])
print("and the best gamma is", GAMMA[np.argmin(SGMBatch_HP)%3])
print("For SAG the best learning rate is", TAU[np.argmin(SAG_HP)//3])
print("and the best gamma is", TAU[np.argmin(SAG_HP)%3])
print("For SG Momentum")

For gradient descent the best learning rate is 0.1
For ADAM the best learning rate is 0.001
[[183. 183. 183.]
 [183. 183. 183.]
 [183. 183. 183.]]
For stochastic GD the best learning rate is 0.001
For stochastic mini batch the best learning rate is 0.1
and the best gamma is 0.001
For SAG the best learning rate is 0.1
%%!and the best gamma is 0.001


In [21]:
help(np.argmin)

Help on function argmin in module numpy.core.fromnumeric:

argmin(a, axis=None, out=None)
    Returns the indices of the minimum values along an axis.
    
    Parameters
    ----------
    a : array_like
        Input array.
    axis : int, optional
        By default, the index is into the flattened array, otherwise
        along the specified axis.
    out : array, optional
        If provided, the result will be inserted into this array. It should
        be of the appropriate shape and dtype.
    
    Returns
    -------
    index_array : ndarray of ints
        Array of indices into the array. It has the same shape as `a.shape`
        with the dimension along `axis` removed.
    
    See Also
    --------
    ndarray.argmin, argmax
    amin : The minimum value along a given axis.
    unravel_index : Convert a flat index into an index tuple.
    
    Notes
    -----
    In case of multiple occurrences of the minimum values, the indices
    corresponding to the first occurrence ar